In [1]:
import pandas as pd
import numpy as np

In [10]:
# 读取数据(如果想要读取自己的记得更改文件路径)
data = pd.read_csv('C:/Users/86187/Downloads/house.csv')
data.head()

C:\Users\86187\AppData\Local\Temp\ipykernel_14800\229430158.py:2: DtypeWarning: Columns (1,11,12,14) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('C:/Users/86187/Downloads/house.csv')


,url,id,Lng,Lat,Cid,tradeTime,DOM,followers,totalPrice,price,...,buildingType,constructionTime,renovationCondition,buildingStructure,ladderRatio,elevator,fiveYearsProperty,subway,district,communityAverage
0,https://bj.lianjia.com/chengjiao/101084782030....,101085000000.0,116.475489,40.019520,1.111030e+12,2016/8/9,1464.0,106,415.0,31680,...,1.0,2005,3,6,0.217,1.0,0.0,1.0,7,56021.0
1,https://bj.lianjia.com/chengjiao/101086012217....,101086000000.0,116.453917,39.881534,1.111030e+12,2016/7/28,903.0,126,575.0,43436,...,1.0,2004,4,6,0.667,1.0,1.0,0.0,7,71539.0
2,https://bj.lianjia.com/chengjiao/101086041636....,101086000000.0,116.561978,39.877145,1.111040e+12,2016/12/11,1271.0,48,1030.0,52021,...,4.0,2005,3,6,0.500,1.0,0.0,0.0,7,48160.0
3,https://bj.lianjia.com/chengjiao/101086406841....,101086000000.0,116.438010,40.076114,1.111040e+12,2016/9/30,965.0,138,297.5,22202,...,1.0,2008,1,6,0.273,1.0,0.0,0.0,6,51238.0
4,https://bj.lianjia.com/chengjiao/101086920653....,101087000000.0,116.428392,39.886229,1.111030e+12,2016/8/28,927.0,286,392.0,48396,...,4.0,1960,2,2,0.333,0.0,1.0,1.0,1,62588.0


In [11]:
# 数据规格(统共318851行26列)
data.shape

(318851, 26)

In [12]:
data.columns

Index(['url', 'id', 'Lng', 'Lat', 'Cid', 'tradeTime', 'DOM', 'followers',
       'totalPrice', 'price', 'square', 'livingRoom', 'drawingRoom', 'kitchen',
       'bathRoom', 'floor', 'buildingType', 'constructionTime',
       'renovationCondition', 'buildingStructure', 'ladderRatio', 'elevator',
       'fiveYearsProperty', 'subway', 'district', 'communityAverage'],
      dtype='object')

In [ ]:
# 扔掉url、id、DOM、totalPrice
data = data.drop(['url', 'id', 'DOM','totalPrice','fiveYearsProperty'], axis=1)